In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# manage ray's relative imports

import ray
runtime_env = {"working_dir": ".." }
ray.init(runtime_env=runtime_env, dashboard_port=13065, include_dashboard=True, dashboard_host="0.0.0.0")
from ray import tune
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune import JupyterNotebookReporter

# manage beams's relative imports
import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment, Study
from src.beam import UniversalDataset, UniversalBatchSampler
from src.beam import Algorithm
from src.beam import LinearNet
from torchvision import transforms
import matplotlib.pyplot as plt

from src.beam import DataTensor
from src.beam.utils import is_notebook
from cifar10_example import Cifar10Network, CIFAR10Algorithm, CIFAR10Dataset

from ray.tune.suggest.hebo import HEBOSearch

2022-07-06 20:35:57,004	INFO services.py:1456 -- View the Ray dashboard at http://172.17.0.2:13065
2022-07-06 20:35:58,022	INFO packaging.py:388 -- Creating a file package for local directory '..'.
2022-07-06 20:35:58,279	INFO packaging.py:241 -- Pushing file package 'gcs://_ray_pkg_c2239984b08464ed.zip' (1.24MiB) to Ray cluster...
2022-07-06 20:35:58,307	INFO packaging.py:243 -- Successfully pushed file package 'gcs://_ray_pkg_c2239984b08464ed.zip'.


In [2]:
path_to_data = '/localdata/elads/data/datasets/cifar10'
root_dir = '/localdata/elads/data/cifar10'

# path_to_data = '/home/shared/data/dataset/cifar10'
# root_dir = '/home/shared/data/results/cifar10'

## Training with a single worker

In [12]:
# here you put all actions which are performed only once before initializing the workers
# for example, setti`ng running arguments and experiment:

args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=0 --amp --lr-d=1e-2 --batch-size=512",
                      "--n-epochs=40 --clip-gradient=1000 --parallel=0 --accumulate=1 --no-deterministic",
                      "--weight-decay=3e-5 --beta1=0.9", 
                      path_to_data=path_to_data, dropout=.0, activation='relu', channels=1024, label_smoothing=.2,
                      padding=6, gain=.2, turn_point=512, final_point=4096, minimal_gain=.05, temperature=0.05)

experiment = Experiment(args)

2022-07-06 20:43:29 | INFO     | Creating new experiment
2022-07-06 20:43:29 | INFO     | Experiment directory is: /localdata/elads/data/cifar10/cifar10/CIFAR10Algorithm/debug/0159_20220706_204329
2022-07-06 20:43:30 | INFO     | beam project: cifar10
2022-07-06 20:43:30 | INFO     | Experiment Hyperparameters
2022-07-06 20:43:30 | INFO     | project_name: cifar10
2022-07-06 20:43:30 | INFO     | algorithm: CIFAR10Algorithm
2022-07-06 20:43:30 | INFO     | identifier: debug
2022-07-06 20:43:30 | INFO     | mp_port: random
2022-07-06 20:43:30 | INFO     | root_dir: /localdata/elads/data/cifar10
2022-07-06 20:43:30 | INFO     | reload: False
2022-07-06 20:43:30 | INFO     | resume: -1
2022-07-06 20:43:30 | INFO     | override: False
2022-07-06 20:43:30 | INFO     | cpu_workers: 0
2022-07-06 20:43:30 | INFO     | device: 0
2022-07-06 20:43:30 | INFO     | parallel: 0
2022-07-06 20:43:30 | INFO     | tensorboard: True
2022-07-06 20:43:30 | INFO     | lognet: True
2022-07-06 20:43:30 | INFO

In [16]:
experiment = Experiment.reload_from_path('/localdata/elads/data/cifar10/cifar10/CIFAR10Algorithm/debug/0159_20220706_204329')

2022-07-06 20:48:44 | INFO     | Reload experiment from path: /localdata/elads/data/cifar10/cifar10/CIFAR10Algorithm/debug/0159_20220706_204329
2022-07-06 20:48:44 | INFO     | Resuming existing experiment
2022-07-06 20:48:44 | INFO     | beam project: cifar10
2022-07-06 20:48:44 | INFO     | Experiment Hyperparameters
2022-07-06 20:48:44 | INFO     | project_name: cifar10
2022-07-06 20:48:44 | INFO     | algorithm: CIFAR10Algorithm
2022-07-06 20:48:44 | INFO     | identifier: debug
2022-07-06 20:48:44 | INFO     | mp_port: random
2022-07-06 20:48:44 | INFO     | root_dir: /localdata/elads/data/cifar10
2022-07-06 20:48:44 | INFO     | reload: True
2022-07-06 20:48:44 | INFO     | resume: 0159_20220706_204329
2022-07-06 20:48:44 | INFO     | override: False
2022-07-06 20:48:44 | INFO     | cpu_workers: 0
2022-07-06 20:48:44 | INFO     | device: 0
2022-07-06 20:48:44 | INFO     | parallel: 0
2022-07-06 20:48:44 | INFO     | tensorboard: True
2022-07-06 20:48:44 | INFO     | lognet: True


In [ ]:
alg = experiment.fit(CIFAR10Algorithm, CIFAR10Dataset, tensorboard_arguments={'images': {'sample': {'dataformats': 'NCHW'}}})

2022-07-06 20:48:50 | INFO     | Single worker mode
2022-07-06 20:48:50 | INFO     | Worker: 1/1 is running...
2022-07-06 20:48:50 | INFO     | Reload experiment from checkpoint: /localdata/elads/data/cifar10/cifar10/CIFAR10Algorithm/debug/0159_20220706_204329/checkpoints/checkpoint_000015


train:   0%|          | 0/73 [00:00<?, ?it/s]

validation:   0%|          | 0/22 [00:00<?, ?it/s]

2022-07-06 20:49:02 | INFO     | 
2022-07-06 20:49:02 | INFO     | Finished epoch 1/40:
2022-07-06 20:49:02 | INFO     | train:
2022-07-06 20:49:02 | INFO     | loss:           | avg: 613.4     | std: 16.78     | min: 577.2     | 25%: 601.5     | 50%: 614.0     | 75%: 621.9     | max: 667.8     
2022-07-06 20:49:02 | INFO     | acc:            | avg: 0.8465    | std: 0.02106   | min: 0.7812    | 25%: 0.834     | 50%: 0.8496    | 75%: 0.8613    | max: 0.8926    
2022-07-06 20:49:02 | INFO     | lr:             | avg: 0.0005    | std: nan       | min: 0.0005    | 25%: 0.0005    | 50%: 0.0005    | 75%: 0.0005    | max: 0.0005    
2022-07-06 20:49:02 | INFO     | validation:
2022-07-06 20:49:02 | INFO     | loss:           | avg: 611.1     | std: 9.262     | min: 597.8     | 25%: 603.9     | 50%: 609.8     | 75%: 617.0     | max: 629.8     
2022-07-06 20:49:02 | INFO     | acc:            | avg: 0.8459    | std: 0.01394   | min: 0.8203    | 25%: 0.8354    | 50%: 0.8477    | 75%: 0.8516    

train:   0%|          | 0/73 [00:00<?, ?it/s]

validation:   0%|          | 0/22 [00:00<?, ?it/s]

2022-07-06 20:49:21 | INFO     | 
2022-07-06 20:49:21 | INFO     | Finished epoch 2/40:
2022-07-06 20:49:21 | INFO     | train:
2022-07-06 20:49:21 | INFO     | loss:           | avg: 590.8     | std: 12.92     | min: 561.9     | 25%: 581.7     | 50%: 589.4     | 75%: 600.8     | max: 624.2     
2022-07-06 20:49:21 | INFO     | acc:            | avg: 0.8701    | std: 0.01719   | min: 0.8281    | 25%: 0.8574    | 50%: 0.8711    | 75%: 0.8828    | max: 0.9121    
2022-07-06 20:49:21 | INFO     | lr:             | avg: 0.0005703 | std: nan       | min: 0.0005703 | 25%: 0.0005703 | 50%: 0.0005703 | 75%: 0.0005703 | max: 0.0005703 
2022-07-06 20:49:21 | INFO     | validation:
2022-07-06 20:49:21 | INFO     | loss:           | avg: 612.5     | std: 10.3      | min: 596.8     | 25%: 604.4     | 50%: 614.9     | 75%: 618.9     | max: 630.5     
2022-07-06 20:49:21 | INFO     | acc:            | avg: 0.8453    | std: 0.01637   | min: 0.8145    | 25%: 0.8379    | 50%: 0.8457    | 75%: 0.8535    

train:   0%|          | 0/73 [00:00<?, ?it/s]

## Training with 4 workers

In [6]:
# here you put all actions which are performed only once before initializing the workers
# for example, setti`ng running arguments and experiment:

args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=0 --half --lr-d=1e-3 --batch-size=512",
                      "--n-epochs=40 --clip-gradient=1000 --parallel=4 --accumulate=1 --no-deterministic",
                      "--weight-decay=1e-5 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, dropout=.0, activation='relu', channels=512, label_smoothing=.2,
                      padding=6, gain=.2, turn_point=1024, final_point=3000, minimal_gain=.05, temperature=1)

experiment = Experiment(args)

2022-07-06 12:28:01 | INFO     | Creating new experiment
2022-07-06 12:28:01 | INFO     | Experiment directory is: /localdata/elads/data/cifar10/cifar10/CIFAR10Algorithm/debug/0147_20220706_122801
2022-07-06 12:28:02 | INFO     | beam project: cifar10
2022-07-06 12:28:02 | INFO     | Experiment Hyperparameters
2022-07-06 12:28:02 | INFO     | project_name: cifar10
2022-07-06 12:28:02 | INFO     | algorithm: CIFAR10Algorithm
2022-07-06 12:28:02 | INFO     | identifier: debug
2022-07-06 12:28:02 | INFO     | mp_port: random
2022-07-06 12:28:02 | INFO     | root_dir: /localdata/elads/data/cifar10
2022-07-06 12:28:02 | INFO     | reload: False
2022-07-06 12:28:02 | INFO     | resume: -1
2022-07-06 12:28:02 | INFO     | override: False
2022-07-06 12:28:02 | INFO     | cpu_workers: 0
2022-07-06 12:28:02 | INFO     | device: 0
2022-07-06 12:28:02 | INFO     | parallel: 4
2022-07-06 12:28:02 | INFO     | tensorboard: True
2022-07-06 12:28:02 | INFO     | lognet: True
2022-07-06 12:28:02 | INFO

In [7]:
alg = experiment.fit(CIFAR10Algorithm, CIFAR10Dataset, tensorboard_arguments={'images': {'sample': {'dataformats': 'NCHW'}}})

2022-07-06 12:28:07 | INFO     | Initializing 4 parallel workers
2022-07-06 12:28:07 | INFO     | Multiprocessing port is: 45081
2022-07-06 12:28:10 | INFO     | Worker: 3/4 is running...
2022-07-06 12:28:10 | INFO     | Worker: 4/4 is running...
2022-07-06 12:28:10 | INFO     | Worker: 1/4 is running...
2022-07-06 12:28:10 | INFO     | Worker: 2/4 is running...


[W socket.cpp:558] [c10d] The client socket has failed to connect to [localhost]:45081 (errno: 101 - Network is unreachable).
[W socket.cpp:558] [c10d] The client socket has failed to connect to [localhost]:45081 (errno: 101 - Network is unreachable).
validation: 100%|██████████| 22/22 [00:00<00:00, 36.45it/s]


2022-07-06 12:28:32 | INFO     | Finished epoch 1/40:
2022-07-06 12:28:32 | INFO     | train:
2022-07-06 12:28:32 | INFO     | loss:	| avg: inf      	|  std: nan      	|  min: 993.0    	|  25%: 1.099e+03	|  50%: 1.308e+03	|  75%: 1.602e+03	|  max: inf      
2022-07-06 12:28:32 | INFO     | acc:	| avg: 0.2357   	|  std: 0.08556  	|  min: 0.08398  	|  25%: 0.1621   	|  50%: 0.2227   	|  75%: 0.3125   	|  max: 0.4141   
2022-07-06 12:28:32 | INFO     | lr:	| avg: 5e-05    	|  std: nan      	|  min: 5e-05    	|  25%: 5e-05    	|  50%: 5e-05    	|  75%: 5e-05    	|  max: 5e-05    
2022-07-06 12:28:32 | INFO     | validation:
2022-07-06 12:28:32 | INFO     | loss:	| avg: 1.029e+03	|  std: 24.41    	|  min: 991.0    	|  25%: 1.013e+03	|  50%: 1.026e+03	|  75%: 1.042e+03	|  max: 1.078e+03
2022-07-06 12:28:32 | INFO     | acc:	| avg: 0.3977   	|  std: 0.01789  	|  min: 0.3711   	|  25%: 0.3853   	|  50%: 0.3975   	|  75%: 0.4062   	|  max: 0.4453   



train:   0%|          | 0/73 [00:00<?, ?it/s]

validation: 100%|██████████| 22/22 [00:00<00:00, 37.24it/s]


2022-07-06 12:28:48 | INFO     | Finished epoch 2/40:
2022-07-06 12:28:48 | INFO     | train:
2022-07-06 12:28:48 | INFO     | loss:	| avg: 924.1    	|  std: 61.74    	|  min: 812.5    	|  25%: 880.0    	|  50%: 919.5    	|  75%: 965.5    	|  max: 1.06e+03 
2022-07-06 12:28:48 | INFO     | acc:	| avg: 0.4973   	|  std: 0.06777  	|  min: 0.3652   	|  25%: 0.4473   	|  50%: 0.5059   	|  75%: 0.5488   	|  max: 0.6426   
2022-07-06 12:28:48 | INFO     | lr:	| avg: 5e-05    	|  std: nan      	|  min: 5e-05    	|  25%: 5e-05    	|  50%: 5e-05    	|  75%: 5e-05    	|  max: 5e-05    
2022-07-06 12:28:48 | INFO     | validation:
2022-07-06 12:28:48 | INFO     | loss:	| avg: 863.6    	|  std: 20.77    	|  min: 838.0    	|  25%: 848.2    	|  50%: 857.2    	|  75%: 874.5    	|  max: 908.0    
2022-07-06 12:28:48 | INFO     | acc:	| avg: 0.5621   	|  std: 0.02601  	|  min: 0.5176   	|  25%: 0.5439   	|  50%: 0.5596   	|  75%: 0.5796   	|  max: 0.6172   


...


2022-07-06 12:38:15 | INFO     | Finished epoch 38/40:
2022-07-06 12:38:15 | INFO     | train:
2022-07-06 12:38:15 | INFO     | loss:	| avg: 455.1    	|  std: 2.917    	|  min: 451.0    	|  25%: 453.0    	|  50%: 454.0    	|  75%: 456.5    	|  max: 463.0    
2022-07-06 12:38:15 | INFO     | acc:	| avg: 0.999    	|  std: 0.001597 	|  min: 0.9941   	|  25%: 0.998    	|  50%: 1.0      	|  75%: 1.0      	|  max: 1.0      
2022-07-06 12:38:15 | INFO     | lr:	| avg: 5e-05    	|  std: nan      	|  min: 5e-05    	|  25%: 5e-05    	|  50%: 5e-05    	|  75%: 5e-05    	|  max: 5e-05    
2022-07-06 12:38:15 | INFO     | validation:
2022-07-06 12:38:15 | INFO     | loss:	| avg: 537.1    	|  std: 9.271    	|  min: 519.5    	|  25%: 534.0    	|  50%: 537.8    	|  75%: 541.9    	|  max: 556.0    
2022-07-06 12:38:15 | INFO     | acc:	| avg: 0.9102   	|  std: 0.00847  	|  min: 0.8926   	|  25%: 0.9048   	|  50%: 0.9102   	|  75%: 0.9175   	|  max: 0.9258   




validation: 100%|██████████| 22/22 [00:00<00:00, 37.53it/s]


2022-07-06 12:38:30 | INFO     | Finished epoch 39/40:
2022-07-06 12:38:30 | INFO     | train:
2022-07-06 12:38:30 | INFO     | loss:	| avg: 454.2    	|  std: 2.319    	|  min: 451.0    	|  25%: 452.5    	|  50%: 453.5    	|  75%: 455.5    	|  max: 463.5    
2022-07-06 12:38:30 | INFO     | acc:	| avg: 0.9996   	|  std: 0.0009898	|  min: 0.9961   	|  25%: 1.0      	|  50%: 1.0      	|  75%: 1.0      	|  max: 1.0      
2022-07-06 12:38:30 | INFO     | lr:	| avg: 5e-05    	|  std: nan      	|  min: 5e-05    	|  25%: 5e-05    	|  50%: 5e-05    	|  75%: 5e-05    	|  max: 5e-05    
2022-07-06 12:38:30 | INFO     | validation:
2022-07-06 12:38:30 | INFO     | loss:	| avg: 539.2    	|  std: 10.83    	|  min: 515.0    	|  25%: 535.2    	|  50%: 539.5    	|  75%: 545.9    	|  max: 561.5    
2022-07-06 12:38:30 | INFO     | acc:	| avg: 0.9103   	|  std: 0.01261  	|  min: 0.877    	|  25%: 0.9062   	|  50%: 0.9092   	|  75%: 0.916    	|  max: 0.9395   




validation: 100%|██████████| 22/22 [00:00<00:00, 37.40it/s]


2022-07-06 12:38:46 | INFO     | Finished epoch 40/40:
2022-07-06 12:38:46 | INFO     | train:
2022-07-06 12:38:46 | INFO     | loss:	| avg: 454.4    	|  std: 2.115    	|  min: 451.8    	|  25%: 452.8    	|  50%: 454.0    	|  75%: 455.8    	|  max: 460.0    
2022-07-06 12:38:46 | INFO     | acc:	| avg: 0.9992   	|  std: 0.001254 	|  min: 0.9941   	|  25%: 0.998    	|  50%: 1.0      	|  75%: 1.0      	|  max: 1.0      
2022-07-06 12:38:46 | INFO     | lr:	| avg: 5e-05    	|  std: nan      	|  min: 5e-05    	|  25%: 5e-05    	|  50%: 5e-05    	|  75%: 5e-05    	|  max: 5e-05    
2022-07-06 12:38:46 | INFO     | validation:
2022-07-06 12:38:46 | INFO     | loss:	| avg: 539.4    	|  std: 9.066    	|  min: 523.5    	|  25%: 531.6    	|  50%: 541.2    	|  75%: 548.0    	|  max: 551.0    
2022-07-06 12:38:46 | INFO     | acc:	| avg: 0.9118   	|  std: 0.01166  	|  min: 0.8945   	|  25%: 0.9014   	|  50%: 0.9131   	|  75%: 0.9194   	|  max: 0.9355   
2022-07-06 12:38:47 | INFO     | Reload exp

In [5]:
np.mean(alg.evaluate('test')['scalar']['acc'])

test:   0%|          | 0/20 [00:00<?, ?it/s]

0.7747989445924759

## Show tensorboard

In [10]:
%load_ext tensorboard

In [11]:
%tensorboard --logdir /localdata/elads/data/cifar10/cifar10/CIFAR10Algorithm/debug/0147_20220706_122801 --port=13067 --bind_all

## Hyperparameter search with native optuna

In [ ]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=0 --half --lr-d=1e-3 --batch-size=512",
                      "--n-epochs=40 --clip-gradient=1000 --parallel=4 --accumulate=1 --no-deterministic",
                      "--weight-decay=1e-5 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, dropout=.0, activation='relu', channels=512, label_smoothing=.2,
                      padding=6, gain=.2, turn_point=1024, final_point=3000, minimal_gain=.05, temperature=1)

study = Study(cifar10_algorithm_generator, args)

In [14]:
def suggest(trial):
    lr = trial.suggest_loguniform("lr", 1e-3, 2e-2)
    print('My suggestion')
    print(lr)
    return {'lr_dense': lr}
    

In [15]:
study.optuna(suggest, direction='maximize', n_jobs=1, n_trials=10)

[I 2022-06-20 08:46:21,221] A new study created in memory with name: cifar10/CIFAR10Algorithm/debug_hp_optimization_20220620_084618


My suggestion
0.009781018192999246
2022-06-20 08:46:21.224 | INFO     | src.beam.experiment:runner_optuna:114 - Next Hyperparameter suggestion:
2022-06-20 08:46:21.225 | INFO     | src.beam.experiment:runner_optuna:116 - lr_dense: 0.009781018192999246
2022-06-20 08:46:21.229 | INFO     | src.beam.experiment:__init__:262 - Creating new experiment
2022-06-20 08:46:21.229 | INFO     | src.beam.experiment:__init__:277 - Experiment directory is: /home/shared/data/results/cifar10/cifar10/CIFAR10Algorithm/debug_hp_optimization_20220620_084618/0000_20220620_084621
2022-06-20 08:46:21.239 | INFO     | src.beam.experiment:run:554 - Single worker mode
2022-06-20 08:46:21.240 | INFO     | src.beam.experiment:run_worker:48 - Worker: 1/1 is running...
0.08
0.16
0.24
0.32
0.4
0.38857142857142857
0.37714285714285717
0.3657142857142857
0.3542857142857143
0.34285714285714286
0.33142857142857146
0.32
0.3085714285714286


KeyboardInterrupt: 

## Hyperparameter search with ray-tune and optuna

In [3]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=0 --amp --lr-d=1e-3 --batch-size=512",
                      "--n-epochs=40 --clip-gradient=1000 --parallel=0 --accumulate=1 --no-deterministic",
                      "--weight-decay=1e-5 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, dropout=.0, activation='relu', channels=512, label_smoothing=.2,
                      padding=6, gain=.2, turn_point=1024, final_point=3000, minimal_gain=.05, temperature=1)

study = Study(args, Alg=CIFAR10Algorithm, Dataset=CIFAR10Dataset, print_results=True)

2022-07-06 16:20:20 | INFO     | Hyperparameter Optimization
2022-07-06 16:20:20 | INFO     | beam project: cifar10
2022-07-06 16:20:20 | INFO     | Experiment Hyperparameters
2022-07-06 16:20:20 | INFO     | project_name: cifar10
2022-07-06 16:20:20 | INFO     | algorithm: CIFAR10Algorithm
2022-07-06 16:20:20 | INFO     | identifier: debug_hp_optimization_20220706_162020
2022-07-06 16:20:20 | INFO     | mp_port: random
2022-07-06 16:20:20 | INFO     | root_dir: /localdata/elads/data/cifar10
2022-07-06 16:20:20 | INFO     | reload: False
2022-07-06 16:20:20 | INFO     | resume: -1
2022-07-06 16:20:20 | INFO     | override: False
2022-07-06 16:20:20 | INFO     | cpu_workers: 0
2022-07-06 16:20:20 | INFO     | device: 0
2022-07-06 16:20:20 | INFO     | parallel: 0
2022-07-06 16:20:20 | INFO     | tensorboard: True
2022-07-06 16:20:20 | INFO     | lognet: True
2022-07-06 16:20:20 | INFO     | deterministic: False
2022-07-06 16:20:20 | INFO     | scale_epoch_by_batch_size: True
2022-07-06 

In [ ]:
# hebo = HEBOSearch(metric="mean_accuracy", mode="max")

# analysis = study.tune(config={"lr_dense": tune.loguniform(1e-3, 2e-2),
#                               "weight_decay": tune.loguniform(1e-6, 1e-4),
#                               "gamma": tune.loguniform(.1, .9),
#                               "dropout": tune.uniform(0, .75),
#                               "scale_down": tune.uniform(0.4, .7),
#                               "scale_up": tune.uniform(0.9, 1.2),
#                               "ratio_down": tune.uniform(0.7, .95),
#                               "ratio_up": tune.uniform(1.05, 1.4),
#                               "channels": tune.choice([128, 256, 512]),
#                               "batch_size": tune.choice([512, 1024, 2048]),},
#                        metric="mean_accuracy",
#                        max_concurrent_trials=4,
#                        resources_per_trial={"gpu": 1},
#                        mode="max",
#                        search_alg=hebo,
#                       progress_reporter=JupyterNotebookReporter(overwrite=True),
#                        num_samples=400)


analysis = study.tune(config={"lr_dense": tune.loguniform(1e-4, 2e-2),
                              "weight_decay": tune.loguniform(1e-6, 1e-2),
                              "beta1": tune.loguniform(.85, .95),
                              "gain": tune.uniform(0.2, .2),
                              "temperature": tune.uniform(0.05, 2.),
                              "minimal_gain": tune.loguniform(.01, .1),
                              "channels": tune.choice([256, 512, 1024]),
                              "padding": tune.choice([4, 6, 8]),
                              "turn_point": tune.choice([256, 512, 1024]),
                              "final_point": tune.choice([2048, 2048*1.5, 2048*2]),
                              "batch_size": tune.choice([256, 512, 1024]),
                              "activation": tune.choice(['relu', 'celu', 'gelu']),},
                       metric="mean_accuracy",
                       max_concurrent_trials=8,
                       resources_per_trial={"gpu": 1},
                       mode="max",
                       search_alg=OptunaSearch(),
                      progress_reporter=JupyterNotebookReporter(overwrite=True),
                       num_samples=400)


Trial name,status,loc,activation,batch_size,beta1,channels,final_point,gain,lr_dense,minimal_gain,padding,temperature,turn_point,weight_decay,acc,iter,total time (s)
runner_tune_ec1791a4,RUNNING,172.17.0.2:39938,relu,512,0.928183,1024,4096,0.2,0.00671536,0.0357185,4,0.062858,512,4.63743e-05,0.934482,35,454.552
runner_tune_3d2a6602,RUNNING,172.17.0.2:40084,relu,512,0.911644,1024,4096,0.2,0.0071578,0.0349454,4,0.0630833,512,3.41031e-05,0.930131,25,308.486
runner_tune_93f28ee2,RUNNING,172.17.0.2:40184,relu,512,0.911794,1024,4096,0.2,0.011317,0.0469718,4,0.0575666,512,2.46689e-05,0.925337,17,217.496
runner_tune_c8115550,RUNNING,172.17.0.2:40344,relu,256,0.927557,1024,4096,0.2,0.0106841,0.0470998,4,0.0569684,512,3.21594e-05,0.763281,2,34.0246
runner_tune_38247390,PENDING,,relu,512,0.927516,1024,4096,0.2,0.00689768,0.0473985,4,0.0550966,512,2.76028e-05,,,
runner_tune_c944283c,TERMINATED,172.17.0.2:23692,celu,256,0.904816,256,2048,0.2,0.0099276,0.0735093,8,1.58846,512,3.5821e-06,0.880295,40,204.667
runner_tune_cc0e4228,TERMINATED,172.17.0.2:23729,relu,512,0.883576,512,2048,0.2,0.000335789,0.01734,4,0.221518,256,6.57065e-05,0.908114,40,233.51
runner_tune_cc12cf64,TERMINATED,172.17.0.2:23731,gelu,1024,0.933654,512,4096,0.2,0.00266318,0.014136,6,1.3378,512,0.00157602,0.910742,40,230.826
runner_tune_cc172366,TERMINATED,172.17.0.2:23733,gelu,512,0.943031,1024,2048,0.2,0.000101818,0.0812117,6,1.14841,1024,2.45449e-06,0.872603,40,497.553
runner_tune_cc1b3c94,TERMINATED,172.17.0.2:24077,relu,256,0.856223,256,4096,0.2,0.00219286,0.0769723,4,0.668636,512,0.00776539,0.907031,40,188.601


In [ ]:
1+1